In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet, CodeInterpreterTool
from user_functions import user_functions

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

with project_client:
    # Initialize agent toolset with user functions and code interpreter
    # [START create_agent_toolset]
    functions = FunctionTool(user_functions)
    code_interpreter = CodeInterpreterTool()

    toolset = ToolSet()
    toolset.add(functions)
    toolset.add(code_interpreter)

    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-assistant",
        instructions="You are a travel assitant that can lookup past flight data for users given the city and time period. First convert city names to airport codes, then use the airport codes to query the flight data. Answer the questions as accurately as possible using the provided functions.",
        toolset=toolset,
    )
    # [END create_agent_toolset]
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Can you tell me about the flight leaving Stockholm 2 days ago from 1PM to 6PM?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    # [START create_and_process_run]
    run = project_client.agents.create_and_process_run(
        thread_id=thread.id, assistant_id=agent.id)
    # [END create_and_process_run]
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

HttpResponseError: (invalid_function_parameters) Invalid schema for function 'get_airport_code_by_city': In context=('properties', 'airports_data'), array schema missing items.
Code: invalid_function_parameters
Message: Invalid schema for function 'get_airport_code_by_city': In context=('properties', 'airports_data'), array schema missing items.

In [3]:
import json
from datetime import datetime

def format_messages(messages):
    """
    Format the messages to look like a conversation and reverse the order of the messages.

    :param messages: The messages to format.
    :return: A formatted string representing the conversation.
    """
    formatted_messages = []
    for message in reversed(messages['data']):
        role = message['role']
        content = message['content'][0]['text']['value']
        formatted_messages.append(f"{role.capitalize()}: {content}")

    return "\n".join(formatted_messages)



print(format_messages(messages))



NameError: name 'messages' is not defined